In [1]:
library(MatchIt)
library(survival)
library(survminer)
library(tidyr)
library(tableone)
library(ggplot2)


Loading required package: ggplot2

Loading required package: ggpubr


Attaching package: ‘survminer’


The following object is masked from ‘package:survival’:

    myeloma




In [ ]:
#Function
survival_curve <- function(data,title,survival_day,expire_flag){
    df_select <- data
    fit <- survfit(Surv(survival_day,expire_flag) ~ groupHPD, data = df_select)

    ggsurvplot(fit, data = df_select, 
               surv.median.line = "hv",  
               conf.int = TRUE, 
              # risk.table = TRUE, 
#               title=title,
               pval = TRUE,
               #add.all = TRUE, 
               xlab = "Follow up time(day)",
               legend = c(0.2,0.2)) 
  #  ggsave(paste0("./plot_survival/plot/",title,".pdf"), print(pp))
}

# 28d mortality 

In [ ]:
df_survival <- read.csv("sk_survival.csv",as.is = TRUE)
df_survival <- as.data.frame(df_survival)
df_select =  df_survival[,c("survival_28day","mortality_28d","groupHPD")]
df_select=as.data.frame(lapply(df_select,as.numeric))

pdf("aumcdb_28d_mortality.pdf")
png("aumcdb_28d_mortality.png")
fit <- survfit(Surv(survival_28day,mortality_28d) ~ groupHPD, data = df_select)
ggsurv <- ggsurvplot(fit, data = df_select, 
               surv.median.line = "hv",  
               conf.int = TRUE, 
               risk.table = TRUE, 
               risk.table.height = 0.25,
               title="AUMC",
               pval = TRUE,pval.size = 7,
               palette = c("#74a8ff","#3fc968","#f8746a"),
               xlab = "Follow up time (days)",
               legend = c(0.8,0.2),
               legend.title = "",legend.labs = c(" DR ", " RR "," PW "), 
               font.main = 25,     
               font.x = 20,        
               font.y = 20,       
               font.tickslab = 16,
               risk.table.fontsize = 6,tables.theme = theme_cleantable()+ 
    theme(axis.text.y = element_text(size = 16))
          )  
ggsurv$plot <- ggsurv$plot +
  theme(legend.text = element_text(size = 16),plot.title = element_text(hjust = 0.5))  
ggsurv$table <- ggsurv$table +
  theme(plot.title = element_text(size = 20, color = "black", face = "bold"))
ggsurv
dev.off()

Warning message in do_once((if (is_R_CMD_check()) stop else warning)("The function xfun::isFALSE() will be deprecated in the future. Please ", :
“The function xfun::isFALSE() will be deprecated in the future. Please consider using base::isFALSE(x) or identical(x, FALSE) instead.”


png 
  2

In [ ]:
# df_survival <- read.csv("sk_survival.csv",as.is = TRUE)
# df_survival <- as.data.frame(df_survival)
# df_select =  df_survival[,c("survival_28day","mortality_28d","groupHPD")]
# df_select=as.data.frame(lapply(df_select,as.numeric))

# #pdf("aumcdb_28d_mortality.pdf")
# fit <- survfit(Surv(survival_28day,mortality_28d) ~ groupHPD, data = df_select)
# ggsurvplot(fit, data = df_select, 
#                surv.median.line = "hv",  
#                conf.int = TRUE, 
#                risk.table = TRUE, 
#                title="AUMC: 28 day-Mortality",
#                palette = c("#74a8ff","#3fc968","#f8746a"),
#                pval = TRUE,
#                #add.all = TRUE, 
#                xlab = "Follow up time(day)",
#                legend = c(0.8,0.2),
#                 legend.title = "",legend.labs = c(" C1 ", " C2 "," C3 ")) 
# #dev.off()

In [5]:
stop!

ERROR: Error in parse(text = x, srcfile = src): <text>:1:5: unexpected '!'
1: stop!
        ^


# 7d mortality - Unmatched data

In [ ]:
df_survival <- read.csv("sk_survival.csv",as.is = TRUE)
df_survival <- as.data.frame(df_survival)
df_select =  df_survival[,c("survival_7day","mortality_7d","groupHPD")]
df_select=as.data.frame(lapply(df_select,as.numeric))

fit <- survfit(Surv(survival_7day,mortality_7d) ~ groupHPD, data = df_select)
ggsurvplot(fit, data = df_select, 
               surv.median.line = "hv",  
               conf.int = TRUE, 
               risk.table = TRUE, 
               title="7 day-Mortality",
               pval = TRUE,
               #add.all = TRUE, 
               xlab = "Follow up time(day)",
               legend = c(0.8,0.2))

# Data Pairing
- ICU type
- Disease
- Age

In [ ]:
df_match_fea <- read.csv("df_match_fea.csv",as.is = TRUE)
df_match_fea <- as.data.frame(df_match_fea)
df_match_fea

In [ ]:
df_match_fea$group_creteria <- df_match_fea[ , "groupHPD"]
data_match <- matchit(group_creteria~  age + first_careunit + myocardial_infarct + congestive_heart_failure + peripheral_vascular_disease + cerebrovascular_disease + Acidosis + Hyperlipidemia + Hypertension + diabetes_mellitus + respiratory_failure + septic_shock + liver_disease, data = df_match_fea, method="nearest", ratio=1, distance = 'logit',caliper = 0.2)
d1 <- data_match$match.matrix
d2 <- rownames(d1)
rownames(d1) <- NULL
pairs <- cbind(d2, d1) # pair represents the treatment group: The number in the control group indicates the index of the data frame
pairs <- as.data.frame(pairs) # pairs[which(pairs$d2 %in% 10007),]
df_match_byClass <- match.data(data_match)
print(summary(data_match)$nn)